Getting the Dataset


In [8]:
!wget https://github.com/Fariaz99cuet/Character_level_GPT/blob/main/lotr.txt

--2024-04-14 10:57:33--  https://github.com/Fariaz99cuet/Character_level_GPT/blob/main/lotr.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘lotr.txt.1’

lotr.txt.1              [ <=>                ] 143.97K  --.-KB/s    in 0.008s  

2024-04-14 10:57:33 (17.8 MB/s) - ‘lotr.txt.1’ saved [147422]



In [2]:
import torch

Config  Parameters


In [130]:
max_iters=10000
eval_iters=200
learning_rate = 1e-3
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
eval_interval = 100

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Lord of the rings.txt

```



In [16]:
with open('lotr.txt.1','r', encoding ='utf-8') as f:

  input =f.read()

In [69]:
print("Length of dataset:",len(input))

Length of dataset: 2096842


In [71]:
#looking at first 500 characters
print(input[:500])

####-SPECIAL NOTE: 

 
In this reprint several minor inaccuracies, most of them noted by readers, have 
been corrected. For example, the rune text now corresponds exactly with the runes 
on  Thror's  Map.  More  important  is  the  matter  of  Chapter  Five.  There  the  true 
story  of  the  ending  of  the  Riddle  Game,  as  it  was  eventually  revealed  (under 
pressure)  by Bilbo  to Gandalf,  is  now  given  according  to  the Red Book,  in  place 
of  the  version  Bilbo  first  gave  to


Vocab_size and vocabolary

In [72]:
chars = sorted(list(set(input)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#&'()*,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ_`abcdefghijklmnopqrstuvwxyz}¢¥«®µ»ó–—‘’‚…
98


In [73]:
print(vocab_size)

98


Character Level simple Tokenization


In [124]:
#creating a character to integer mapping
stoi = {ch:i for i,ch in enumerate(chars) }
itos={i:ch for i,ch in enumerate(chars)}
encode = lambda e : [stoi[c] for c in e]
decode = lambda l: ''.join([itos[i] for i in l])



In [125]:
print(encode("When you play the game of thrones, you win or you die"))
print(decode(encode("When you play the game of thrones, you win or you die")))

[52, 65, 62, 71, 1, 82, 72, 78, 1, 73, 69, 58, 82, 1, 77, 65, 62, 1, 64, 58, 70, 62, 1, 72, 63, 1, 77, 65, 75, 72, 71, 62, 76, 10, 1, 82, 72, 78, 1, 80, 66, 71, 1, 72, 75, 1, 82, 72, 78, 1, 61, 66, 62]
When you play the game of thrones, you win or you die


In [101]:
#encoding the entire text and storing it in torch tensor

import torch
data = torch.tensor(encode(input), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

torch.Size([2096842]) torch.int64
tensor([ 4,  4,  4,  4, 11, 48, 45, 34, 32, 38, 30, 41,  1, 43, 44, 49, 34, 24,
         1,  0,  0,  1,  0, 38, 71,  1, 77, 65, 66, 76,  1, 75, 62, 73, 75, 66,
        71, 77,  1, 76, 62, 79, 62, 75, 58, 69,  1, 70, 66, 71, 72, 75,  1, 66,
        71, 58, 60, 60, 78, 75, 58, 60, 66, 62, 76, 10,  1, 70, 72, 76, 77,  1,
        72, 63,  1, 77, 65, 62, 70,  1, 71, 72, 77, 62, 61,  1, 59, 82,  1, 75,
        62, 58, 61, 62, 75, 76, 10,  1, 65, 58, 79, 62,  1,  0, 59, 62, 62, 71,
         1, 60, 72, 75, 75, 62, 60, 77, 62, 61, 12,  1, 35, 72, 75,  1, 62, 81,
        58, 70, 73, 69, 62, 10,  1, 77, 65, 62,  1, 75, 78, 71, 62,  1, 77, 62,
        81, 77,  1, 71, 72, 80,  1, 60, 72, 75, 75, 62, 76, 73, 72, 71, 61, 76,
         1, 62, 81, 58, 60, 77, 69, 82,  1, 80, 66, 77, 65,  1, 77, 65, 62,  1,
        75, 78, 71, 62, 76,  1,  0, 72, 71,  1,  1, 49, 65, 75, 72, 75,  6, 76,
         1,  1, 42, 58, 73, 12,  1,  1, 42, 72, 75, 62,  1,  1, 66, 70, 73, 72,
      

In [102]:
block_size =8
batch_size=4
ix = torch.randint(len(data) - block_size, (batch_size,))
ix

tensor([408484, 439577, 642178, 191201])

In [103]:
 x = torch.stack([data[i:i+block_size] for i in ix])
 x

tensor([[ 0, 31, 66, 69, 59, 72,  1,  1],
        [65, 58, 69, 77,  1,  1, 58, 71],
        [58, 64, 62,  1, 80, 65, 66, 60],
        [ 1,  1, 51, 62, 75, 82,  1,  1]])

In [104]:
#spliting the data into train and validation set

n =int(0.9*len(data))

train = data[:n]

val = data[n:]

In [105]:
block_size = 8
train[:block_size+1]

tensor([ 4,  4,  4,  4, 11, 48, 45, 34, 32])

In [106]:
x =train[:block_size]
y = train[1:block_size+1]
print(x,y)

tensor([ 4,  4,  4,  4, 11, 48, 45, 34]) tensor([ 4,  4,  4, 11, 48, 45, 34, 32])


Getting Batch Data

In [107]:
def get_batch(split):
  data =train if split=="train" else val
  ix=torch.randint(len(data)-block_size,(batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y=x.to(device),y.to(device)
  return x,y

Loss Estimation

In [108]:
eval_iters=200
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [109]:
import torch.nn as nn
from torch.nn import functional as F
n_embed=128

Single Headed Attention

In [110]:
class Head(nn.Module):

  """One headed attention"""

  def __init__(self,head_size):
    super().__init__()
    self.key=nn.Linear(n_embed,head_size,bias=False)
    self.Query=nn.Linear(n_embed,head_size,bias=False)
    self.Value=nn.Linear(n_embed,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

    self.dropout =nn.Dropout(dropout)

  def forward(self,x):
    B,T,C = x.shape
    k = self.key(x) #(B,T,C)
    q = self.Query(x) #(B,T,C)

    wei= q @ k.transpose(-2,-1) ##(B,T,C) @ (B,C,T) = (B,T,T)
    wei= wei.masked_fill(self.tril[:T,:T] ==0,float('-inf')) #(B,T,T)
    wei =F.softmax(wei,dim=-1) #(B,T,T)
    wei=self.dropout(wei)
    v=self.Value(x)#(B,T,C)

    out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)

    return out




Multi Head Attention

In [111]:
class MultiHeadAttention(nn.Module):
  "Multi headed attention "
  def __init__(self,num_head,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_head)])
    self.proj = nn.Linear(n_embed,n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):

    out = torch.cat([h(x) for h in self.heads],dim=-1)

    out = self.dropout(self.proj(out))

    return out

Feed forward network

In [112]:
class FeedForward(nn.Module):

  def __init__(self,n_embed):
    super().__init__()
    self.net =nn.Sequential(
        nn.Linear(n_embed,4*n_embed),
        nn.ReLU(),
        nn.Linear(4*n_embed,n_embed),
        nn.Dropout(dropout),)

  def forward(self,x):
    return self.net(x)




Single Transformer Block

In [113]:
class Block(nn.Module):

  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size=n_embed//n_head
    self.sa=MultiHeadAttention(n_head,head_size)
    self.ffwd=FeedForward(n_embed)
    self.ln1=nn.LayerNorm(n_embed)
    self.ln2=nn.LayerNorm(n_embed)

  def forward(self,x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

Bigram Model for Text Generation

In [122]:
#Bigram model

n_layers =16

class BigramLanguageModel(nn.Module):

  def __init__(self):

     super().__init__()
     #each token reads logits of next token from embedding lookup table

     self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
     self.position_embedding_table=nn.Embedding(block_size,n_embed)
     self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
     self.ln=nn.LayerNorm(n_embed)
     self.ln_head=nn.Linear(n_embed,vocab_size)

  def forward(self,idx,target=None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T,device=device))
    x = tok_emb + pos_emb #(B,T,c)
    x = self.blocks(x)#(B,T,c)
    x = self.ln(x)#(B,T,c)
    logits = self.ln_head(x) #(B,T,vocab_size)

    if target is None:
      loss=None
    else :
      B, T, C = logits.shape

      logits = logits.view(B*T,C)
      target = target.view(B*T)

      loss=F.cross_entropy(logits,target)

    return logits,loss


  def generate(self,idx,max_new_tokens):



    for _ in range(max_new_tokens):

      idx_cond = idx[:,-block_size:]

      logits,loss=self(idx_cond)

      logits = logits[:,-1,:] #(B,C)

      prob = F.softmax(logits,dim=-1)

      idx_next = torch.multinomial(prob,num_samples=1)

      idx=torch.cat((idx,idx_next),dim=1)

    return idx








Training the model

In [131]:

model = BigramLanguageModel()

m=model.to(device)

print(sum(p.numel() for p in model.parameters())/1e6,'M Parameters')

optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):

  if iter % eval_interval==0 or iter==eval_interval-1:

    losses=estimate_loss()
    print(f"step {iter} : train_loss={losses['train']:.4f} val_loss={losses['val']:.4f}")

    x_batch,y_batch=get_batch('train')

    logits,loss=model(x_batch,y_batch)

    optimizer.zero_grad(set_to_none=None)
    loss.backward()
    optimizer.step()

0.82109 M Parameters
step 0 : train_loss=4.6874 val_loss=4.6867
step 99 : train_loss=4.0561 val_loss=4.0812
step 100 : train_loss=3.7114 val_loss=3.7491
step 200 : train_loss=3.5123 val_loss=3.5470
step 300 : train_loss=3.3696 val_loss=3.4070
step 400 : train_loss=3.2674 val_loss=3.3027
step 500 : train_loss=3.1953 val_loss=3.2234
step 600 : train_loss=3.1367 val_loss=3.1730
step 700 : train_loss=3.0875 val_loss=3.1210
step 800 : train_loss=3.0420 val_loss=3.0792
step 900 : train_loss=3.0083 val_loss=3.0467
step 1000 : train_loss=2.9861 val_loss=3.0162
step 1100 : train_loss=2.9622 val_loss=2.9943
step 1200 : train_loss=2.9423 val_loss=2.9772
step 1300 : train_loss=2.9205 val_loss=2.9611
step 1400 : train_loss=2.9034 val_loss=2.9355
step 1500 : train_loss=2.8888 val_loss=2.9222
step 1600 : train_loss=2.8593 val_loss=2.8971
step 1700 : train_loss=2.8508 val_loss=2.8814
step 1800 : train_loss=2.8273 val_loss=2.8645
step 1900 : train_loss=2.8126 val_loss=2.8377
step 2000 : train_loss=2.79

Generation Of text

In [133]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1500)[0].tolist()))



Wes he amecey Oreyhund igy, cad frencl#n trinan  Goun s amen weds-sthes yre   an tive as TboumAve. sthamol thed  t  wanoud  fs  toomamperer thinenthind tro spored, mashaounasong  Bbipe anAd mee6 
s 
 t  s 
 bltoktwee"oukas  in cowiloultro aGaved looeine imey las' Rokevt Gm. ace  cin  nyomut t as  acolingshee he  0s Yevhingeres  s w
uind jom  tee in  rr. I'Erors anitond  "us C thilly owe pebed ge leecEne.   `  havorine d e iced  wacles 
 tey ing thsheculain   mer t thte-oekeat, ind. lik thestaRote" ds toainey d   wind rithted pe t  theupid. und  gurtipee E cougang 
  wero Eyst, tomesoupobilas hid   yelrey toullilik in cupheca>e   inevetachanou.nenin sntot o wane,  
b ! 
uy id, bind 'arim, in, cod fy  then,es &o! aknenitiysby ce in, htupais totostre inareuthe 
s.  Rblkerousnle 
: 
ghe. -  delfindelis myowan,' imroond    lorou ad oed amle 
2or c1veed  ren  
s-/atlalllogband 
ini.deed Ysitgof t waninoug 

: n  fif 
vendet t ce Ce in o  intonthe? louldoulrmy Aplldilband IRinc, oameres as b